In [1]:
import cv2
import numpy as np

In [2]:
# Загрузка двух изображений
img1 = cv2.imread("images/place.jpg")
h, w, _ = img1.shape
#Cлучайная генерация координат для теста
x = np.random.randint(0, w - 300) 
y = np.random.randint(0, h - 200)
img2 = img1[y:y+200,x:x+300]

# Конвертирование изображений в градации серого
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [4]:
cv2.imshow('image 1',img1)
cv2.imshow('image 2',img2)
cv2.waitKey(0)
cv2.destroyAllWindows

<function destroyAllWindows>

In [5]:
# Создание объекта SIFT
sift = cv2.SIFT_create(sigma=0.7)

# Нахождение ключевых точек и дескрипторов на первом изображении
kp1, des1 = sift.detectAndCompute(gray1, None)

# Нахождение ключевых точек и дескрипторов на втором изображении
kp2, des2 = sift.detectAndCompute(gray2, None)

# Нахождение соответствий между дескрипторами двух изображений
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

In [6]:
# Фильтрация соответствий по лучшим
good_matches = []
for m, n in matches:
    #Методом подбора был найден коэффициент 0.2
    if m.distance < 0.2 * n.distance: 
        good_matches.append(m)

img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imwrite("images/Mathces.jpg", img_matches)

True

In [7]:
# Если количество соответствий меньше 4, то считаем, что изображения не связаны
if len(good_matches) < 4:
    print("Images are not connected")
else:
    # Нахождение координат ключевых точек в массивах
    #Ключевые точки изображения №1
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    #Ключевые точки изображения №2
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Нахождение гомографической матрицы
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Нахождение границы второго изображения на первом изображении
    #h, w = img1.shape[:2]
    #pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
    #dst = cv2.perspectiveTransform(pts, M)
    matching_region = cv2.boundingRect(src_pts)

    # Вывод результата
    print("Matching region:", matching_region)
    cv2.rectangle(img1, (matching_region[0], matching_region[1]),
                  (matching_region[0] + matching_region[2], matching_region[1] + matching_region[3]),
                  (0, 255, 0), 2)
    if abs(x-matching_region[0]) > 0.05*w or \
    abs(y-matching_region[1]) > 0.05*h or \
    abs(x+300-(matching_region[0] + matching_region[2])) > 0.05*w or \
    abs(y+200-(matching_region[1] + matching_region[3])) > 0.05*h:
        print("Mistakes too big")
    else:
        print("Good job")
    print(f'Diff \nw-x:{x-matching_region[0]} \nh-y:{y-matching_region[1]} \nw2-x2:{x+300-(matching_region[0] + matching_region[2])} \nh2-y2:{y+200-(matching_region[1] + matching_region[3])}')
    cv2.imshow('resultMatch',img1)
    cv2.waitKey(0)
    cv2.destroyAllWindows
    #cv2.imwrite("images/Matching.jpg", img1)

Matching region: (21, 942, 293, 194)
Good job
Diff w-x:-4 
 h-y:-3 
 w2-x2:3 
 h2-y2:3
